In [72]:
# Python
import pandas as pd
import numpy as np
from fbprophet import Prophet
import matplotlib.pyplot as plt

In [68]:
# 读取文件
df = pd.read_excel('./clear.xlsx',encoding='utf-8',sheet_name=2)
# df.rename(columns={'Sales':'y',"Month":'ds'},inplace=True)
# 读取指定的行，我们只需要日期和电力总量
# df.head()

newdf = pd.DataFrame()
newdf['ds'] = df['日期']
newdf['y'] = df['用电总和']
# 筛选掉0值
# newdf = newdf[~newdf['y'].isin([0])]

# 每一列*1000
newdf['y'] = newdf['y'] * 1000
# newdf.to_csv('./newdf.csv')
newdf.tail(5)

,ds,y
742,2021-01-20,218.64
743,2021-01-21,188.88
744,2021-01-22,193.20
745,2021-01-24,123.12
746,2021-01-25,178.80


In [135]:
# 切分数据集
l = len(newdf)
print(l)
train_data = (newdf.loc[l-60:l-7,:,])
test_data = (newdf.loc[l-7:,:])
# print(train_data)
# print(test_data)
m = Prophet(interval_width=0.9,changepoint_range=0.9,growth = "linear")
m.fit(train_data)

# draw=pd.concat([pd.DataFrame(train_data['y'])],axis=1)
# print(draw)
# draw.iloc[:60-7,0].plot(figsize=(12,6),color="black",label="train")
#draw.iloc[len(draw) - 7 :,1].plot(figsize=(12,6),color="red",label="real")
#draw.iloc[:7,2].plot(figsize=(12,6),color="blue",label="predict")
#plt.legend(('train', 'real','predict'),fontsize='15')
#plt.title("train",fontsize='30') #添加标题
#展示在训练集上的表现 

INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
747


In [138]:
# 数据预测，periods表示预测点数，freq表示预测频率
future = m.make_future_dataframe(periods=11,freq='D') # 预测未来
forecast = m.predict(future)
# m.plot(forecast) # 绘制图形
l = len(forecast)

# print(forecast)
forecast = forecast[['trend']].loc[l-7:,:]
test_data = (test_data[['y']])

# error
f = np.array(forecast)
t = np.array(test_data)
for i in range(0,len(f)):
    print(f[i][0],t[i][0])
    print('mse: ' , (f[i][0] - t[i][0]) / t[i][0])


draw=pd.concat([pd.DataFrame(train_data['y']),pd.DataFrame(test_data['y']),pd.DataFrame(forecast['trend'])],axis=1)
t = draw[['trend']].shift(len(draw) - 7)
draw = pd.concat([pd.DataFrame(train_data['y']),pd.DataFrame(test_data['y']),pd.DataFrame(draw['trend'])],axis=1)
print(draw)
# draw.iloc[:60-7,0].plot(figsize=(12,6),color="black",label="train")
# draw.iloc[len(draw) - 7 :,1].plot(figsize=(12,6),color="red",label="real")
# draw.iloc[:7,2].plot(figsize=(12,6),color="blue",label="predict")
# plt.legend(('train', 'real','predict'),fontsize='15')
# plt.title("train",fontsize='30') #添加标题

154.87025854391342 184.32
mse:  -0.1597750730039419
154.32783514656865 193.2
mse:  -0.20120168143598005
153.7854117492239 218.64000000000001
mse:  -0.2966272788637766
153.2429883518791 188.88000000000002
mse:  -0.18867541109763303
152.70056495453431 193.20000000000002
mse:  -0.20962440499723448
152.15814155718954 123.11999999999996
mse:  0.23585235182902523
151.6157181598448 178.79999999999998
mse:  -0.15203737047066665
      y       y       trend
58  NaN     NaN  154.870259
59  NaN     NaN  154.327835
60  NaN     NaN  153.785412
61  NaN     NaN  153.242988
62  NaN     NaN  152.700565
..   ..     ...         ...
742 NaN  218.64         NaN
743 NaN  188.88         NaN
744 NaN  193.20         NaN
745 NaN  123.12         NaN
746 NaN  178.80         NaN

[67 rows x 3 columns]
